In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch()

## Table Create

In [ ]:
# db 이름 설정
INDEX_NAME = "test_index"

# db 셋팅
INDEX_SETTINGS = {
"settings" : {
    "index":{
    "analysis":{
        "analyzer":{
        "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

        }
        }
    }
    }
},
"mappings": {

    "properties" : {
        "context" : {
        "type" : "text",
        "analyzer": "korean",
        "search_analyzer": "korean"
        }
        
        "title" : {
        "type" : "text",
        "analyzer": "korean",
        "search_analyzer": "korean"
        }
    }

}
}


In [ ]:
def create_index(index):
    if not es.indices.exists(index=index):
        return es.indices.create(index=index)

In [2]:
def insert(body):
    return es.index(index=index, doc_type=doc_type, body=body)


'''
index = 'news'
doc_type = 'daum'
data = {
    'date' : '20200811',
    'category' : 'society',
    'title' : '유령'은 어느 행성에서 지구로 왔는가?'
    'content' : '크립톤 행성'
}

ir = insert(data)
'''

## Table Read

In [ ]:
def search(index, data=None):
    if data is None:
        data = {"match_all" : {}}
    else:
        data = {"match" : data}
    body = {"query" : data}
    res = es.search(index=index, body=body)
    
'''
sr = search(index, {'category': 'society'})
pprint.pprint(sr)
'''

## Table Update

In [ ]:
def update(id, doc):
    body = {'doc' : doc}
    res = es.update(index=index, id=id, body=body, doc_type=doc_type)
    return res

ir = insert(data)
ur = update(ir['_id'], {newspaper:'sbs'})
pprint.pprint(ur)

## Table Delete

In [ ]:
def delete_index(index):
    if not es.indices.exists(index=index):
        return es.indices.delete(index=index)

In [ ]:
def delete_by_id(index, id):
    return es.delete(index, id=id)